In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import libraries
from google.colab import drive
import requests
import pandas as pd
import time

In [ ]:
#Keys
API_KEY = "YourKeys"
CHANNEL_ID = "YourChannelID"

In [ ]:
def get_video_details(video_id):
      #Making a Second API Call & collecting view, like, comment counts
      url_video_stats = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+API_KEY
      response_video_stats = requests.get(url_video_stats).json()
      
      view_count = response_video_stats['items'][0]['statistics']['viewCount']
      like_count = response_video_stats['items'][0]['statistics']['likeCount']
      comment_count = response_video_stats['items'][0]['statistics']['commentCount']

      return view_count, like_count, comment_count

In [ ]:
def get_videos(df):
    #make first API call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=10000"+pageToken #need to read documantation to understand
    
    response = requests.get(url).json()  
    time.sleep(1)#give it a time before starting the for loop

    # take info from all Videos by creating the loop
    for video in response['items']:
      if video['id']['kind'] == "youtube#video":
        video_id = video['id']['videoId']
        video_title = video['snippet']['title']
        video_title = str(video_title).replace(" ","_")
        video_title = str(video_title).replace("💍","diamond")
        upload_date = video['snippet']['publishedAt']
        upload_date = str(upload_date).split("T")[0]

        view_count, like_count, comment_count = get_video_details(video_id)

        # Saving to a Pandas DataFrame
        df = df.append({'video_id':video_id,'video_title':video_title,
                        'upload_date':upload_date,'view_count':view_count,
                        'like_count':like_count,'comment_count':comment_count},
                       ignore_index=True)
    return df

In [ ]:
# building DataFrame
df = pd.DataFrame(columns=["video_id","video_title","upload_date","view_count","like_count","comment_count"])
df = get_videos(df)
df

,video_id,video_title,upload_date,view_count,like_count,comment_count
0,mTL23Gd-T3g,Engagement_Ring_Animation,2022-08-26,54,1,0
1,MEnebfZ_CyQ,Ring_and_band_animation,2022-07-29,71,2,1
2,3TxVeAGJXCQ,Ring_animation_diamond,2022-07-27,115,3,1
3,VKVknZOXDSo,white_gold_Ring,2021-03-02,365,13,1
4,cUEueCDG-xY,Ambiguous_Cylinder_Illusion,2020-07-22,127,6,3
5,uasLj9J56os,swiss_blue_topaz_ring,2020-07-10,114,4,3


In [ ]:
# write csv in GoogleDrive

drive.mount('/drive')
df.to_csv('/drive/My Drive/SyncFolder/projects/API_DB/files/youtube_API_pull.csv')

Mounted at /drive
